In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
top_60_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(60)
top_60_coins_df = top_60_coins_df.reset_index(drop=True)
top_60_coins = top_60_coins_df.get('name')

In [13]:
from constants.coin_names import CoinNames
from settings import DATA_DIR
import pandas as pd
import os
dfs = {}
no_data = []
# for coin in CoinNames.__members__:
for coin in top_60_coins:
    p = DATA_DIR / f'{coin}_15m_2880.csv'
    if os.path.exists(p):
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        dfs[coin] = df
    else:
        no_data.append(coin)
else:
    os.sys.stderr.write(f'No data for {no_data}')

No data for []

In [14]:
dfs["SOL"].iloc[-1, :]["tic"]

1714089600000

In [15]:
from collections import Counter


def get_majority_shape(dfs):
    # Get the shapes of all DataFrames
    shapes = [df.shape for df in dfs.values()]
    
    # Count occurrences of each shape
    shape_counts = Counter(shapes)
    
    # Display votes for each shape
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Votes: {count}")
    
    # Find the most common shape
    majority_shape, majority_votes = shape_counts.most_common(1)[0]
    
    return majority_shape

def get_majority_start_end(dfs):
    # Get the shapes of all DataFrames
    start_end = [(df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"] ) for df in dfs.values()]
    
    # Count occurrences of each shape
    start_end_counts = Counter(start_end)
    
    # Display votes for each shape
    for se, count in start_end_counts.items():
        print(f"Start End: {se}, Votes: {count}")
    
    # Find the most common shape
    majority_se, majority_votes = start_end_counts.most_common(1)[0]
    
    return majority_se

major_shape = get_majority_shape(dfs)
major_se = get_majority_start_end(dfs)

Shape: (2881, 12), Votes: 57
Shape: (2250, 12), Votes: 1
Shape: (1168, 12), Votes: 1
Shape: (231, 12), Votes: 1
Start End: (1711497600000, 1714089600000), Votes: 57
Start End: (1712065500000, 1714089600000), Votes: 1
Start End: (1713039300000, 1714089600000), Votes: 1
Start End: (1713882600000, 1714089600000), Votes: 1


In [16]:
def filter_dfs(dfs, shape, start_end):
    valid_dfs = {}
    for coin, df in dfs.items():
        if df.shape != shape:
            print(f"Shape mismatch for {coin}. Expected {shape}, got {df.shape}")
            continue
        elif (df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"]) != start_end:
            print(f"Start/End mismatch for {coin}. Expected {start_end}, got {df.iloc[0, :]['tic'], df.iloc[-1, :]['tic']}")
            continue
        else:
            valid_dfs[coin] = df

    return valid_dfs


valid_dfs = filter_dfs(dfs, major_shape, major_se)

Shape mismatch for ENA. Expected (2881, 12), got (2250, 12)
Shape mismatch for HBAR. Expected (2881, 12), got (1168, 12)
Shape mismatch for MERL. Expected (2881, 12), got (231, 12)


In [17]:
df = valid_dfs["SOL"]

df.head()

,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,,
SOL,2024-03-27-00:00:00,2024-03-27-00:00:00,2024-03-27-00:14:59,15m,537,189.22,190.63,188.85,190.62,2893.32,1711497600000,1711498499999
SOL,2024-03-27-00:15:00,2024-03-27-00:15:00,2024-03-27-00:29:59,15m,621,190.42,190.49,188.92,189.25,3853.90,1711498500000,1711499399999
SOL,2024-03-27-00:30:00,2024-03-27-00:30:00,2024-03-27-00:44:59,15m,689,192.10,192.39,190.07,190.42,2233.22,1711499400000,1711500299999
SOL,2024-03-27-00:45:00,2024-03-27-00:45:00,2024-03-27-00:59:59,15m,666,192.57,192.60,191.24,192.10,4063.04,1711500300000,1711501199999
SOL,2024-03-27-01:00:00,2024-03-27-01:00:00,2024-03-27-01:14:59,15m,677,192.65,192.82,191.61,192.57,2999.24,1711501200000,1711502099999


In [18]:
_df = pd.concat(valid_dfs.values())
_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164217 entries, BTC to ENS
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   datetime          164217 non-null  object 
 1   start_time        164217 non-null  object 
 2   end_time          164217 non-null  object 
 3   interval          164217 non-null  object 
 4   number_of_trades  164217 non-null  int64  
 5   close             164217 non-null  float64
 6   high              164217 non-null  float64
 7   low               164217 non-null  float64
 8   open              164217 non-null  float64
 9   volume            164217 non-null  float64
 10  tic               164217 non-null  int64  
 11  toc               164217 non-null  int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 16.3+ MB


In [19]:
_df.head()

,datetime,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,,
BTC,2024-03-27-00:00:00,2024-03-27-00:00:00,2024-03-27-00:14:59,15m,500,69959.0,70076.0,69886.0,70049.0,24.21455,1711497600000,1711498499999
BTC,2024-03-27-00:15:00,2024-03-27-00:15:00,2024-03-27-00:29:59,15m,470,70175.0,70183.0,69897.0,69959.0,12.35217,1711498500000,1711499399999
BTC,2024-03-27-00:30:00,2024-03-27-00:30:00,2024-03-27-00:44:59,15m,1014,70543.0,70589.0,70087.0,70176.0,73.40334,1711499400000,1711500299999
BTC,2024-03-27-00:45:00,2024-03-27-00:45:00,2024-03-27-00:59:59,15m,844,70735.0,70795.0,70378.0,70540.0,75.26471,1711500300000,1711501199999
BTC,2024-03-27-01:00:00,2024-03-27-01:00:00,2024-03-27-01:14:59,15m,980,70748.0,70800.0,70481.0,70735.0,70.56705,1711501200000,1711502099999


In [20]:
df = _df

In [21]:
df.to_csv(DATA_DIR / "all_15m_2880.csv", index=True)